In [1]:
import os
import cv2
import numpy as np

In [2]:
Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6
scale = 2

BLOCK_STEP = 16
BLOCK_SIZE = 32

In [37]:
def prepare_data(_path):
    names = os.listdir(_path)   # 100개의 데이터를 순서대로 불러오기
    names = sorted(names)       # 100개의 데이터가 순서대로 정리
    nums = names.__len__()      # 100

    data = np.zeros((nums * Random_Crop, 1, Patch_size, Patch_size), dtype=np.double)
    label = np.zeros((nums * Random_Crop, 1, label_size, label_size), dtype=np.double)

    for i in range(nums):
        name = _path + names[i]
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        shape = hr_img.shape

        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        
        w, h = hr_img.shape
        new_width = int(w / scale)
        new_height = int(h / scale)

        # two resize operation to produce training data and labels
        lr_img = cv2.resize(hr_img, (new_width, new_height))
        lr_img = cv2.resize(lr_img, (w, h))

         # produce Random_Crop random coordinate to crop training img
        Points_x = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)
        Points_y = np.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)

        for j in range(Random_Crop):
            lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]
            hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

            lr_patch = lr_patch.astype(float) / 255.
            hr_patch = hr_patch.astype(float) / 255.

            data[i * Random_Crop + j, 0, :, :] = lr_patch
            label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

    return data, label

In [41]:
datasets = prepare_data('source/')

datasets[1].shape

(420, 1, 20, 20)